In [28]:
import csv
import joblib
import numpy as np


In [29]:
import cv2
from sklearn.cluster import KMeans
import numpy as np
import cv2
from collections import Counter


In [30]:
T=10

# Load Bobot Predictor
a_file = csv.reader(open('./model/bbt_pred.csv'), delimiter=',')
a = np.zeros(shape=T)
for row in a_file:
    for i, ai in enumerate(row[:-1]):
        a[i] = ai

# Load Model Predictor
wl = np.zeros(shape=T, dtype=object)
for i in range(T):
    wl[i] = joblib.load(open("./model/ds_tree_%s.pkl"%i, 'rb'))

print(a, wl)

[1.22377543 0.83681558 1.19817576 1.04277464 1.01224779 0.93203089
 0.93632423 0.40676802 0.61734525 0.8214544 ] [DecisionTreeClassifier(max_depth=1) DecisionTreeClassifier(max_depth=1)
 DecisionTreeClassifier(max_depth=1) DecisionTreeClassifier(max_depth=1)
 DecisionTreeClassifier(max_depth=1) DecisionTreeClassifier(max_depth=1)
 DecisionTreeClassifier(max_depth=1) DecisionTreeClassifier(max_depth=1)
 DecisionTreeClassifier(max_depth=1) DecisionTreeClassifier(max_depth=1)]


In [31]:
def predict(fitur, a, weak_learn):
    # Mendapatkan hipotesis dari weak learn
    y_pred = np.array([wl.predict(fitur) for wl in weak_learn], dtype=int)
    # Hasil Prediksi
    return np.sign(np.dot(a, y_pred))[0]
        

In [32]:
def get_fitur(image_path, number_of_colors):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    modified_image = cv2.resize(image, (600, 400), interpolation = cv2.INTER_AREA)
    modified_image = modified_image.reshape(modified_image.shape[0]*modified_image.shape[1], 3)
    
    clf = KMeans(n_clusters = number_of_colors)
    labels = clf.fit_predict(modified_image)
    
    counts = Counter(labels)
    
    #convert fitur to np array
    fitur = np.zeros((20,1))
    for i in range(len(counts)):
        fitur[i] = counts[i]
        
    return fitur


In [35]:
fitur = get_fitur("yes.png", 20).astype(int)
fitur = fitur.reshape((fitur.shape[0],))
pr = predict([fitur], a, wl)
print('Colorful' if pr > 0 else 'Nocolorful')

Colorful
